In [1]:
import pandas as pd
import numpy as np

train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

train

,latitud,longitud,Barrio,Ciudad,Area_total,Area_constr,MONEDA,Area_constr_cat,NroBanios,Dormitorios,Antiguedad,Cocheras,areas_diff,geohash_grado5,geohash_grado6,geohash_grado7,geohash_grado8,geohash_grado9,Precio_m2_total
0,-12.095549,-77.059553,Magdalena,Lima,260.0,317.0,S/,9.0,4.0,5.0,41.0,3.0,-57.0,6mc5n,6mc5nq,6mc5nqn,6mc5nqnf,6mc5nqnfq,7406.153846
1,-12.283564,-76.847231,Lurín,Lima,1000.0,399.0,USD,13.0,3.0,4.0,10.0,6.0,601.0,6mc65,6mc657,6mc657c,6mc657ce,6mc657ce1,1925.000000
2,-12.068401,-77.091116,San Miguel,Lima,225.0,160.0,S/,2.0,3.0,6.0,27.0,2.0,65.0,6mc5m,6mc5me,6mc5mep,6mc5mepb,6mc5mepbq,5664.000000
3,-12.210306,-77.011977,Chorrillos,Lima,1385.0,490.0,S/,16.0,4.0,5.0,18.0,4.0,895.0,6mc4x,6mc4x9,6mc4x91,6mc4x91t,6mc4x91tf,2433.212996
4,-12.095819,-77.014561,San Isidro,Lima,360.0,450.0,USD,15.0,4.0,5.0,34.0,3.0,-90.0,6mc5p,6mc5pq,6mc5pqp,6mc5pqp8,6mc5pqp8w,8555.555556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4968,-12.131029,-77.008097,Santiago de Surco,Lima,343.0,280.0,USD,7.0,3.0,4.0,29.0,2.0,63.0,6mc4z,6mc4zx,6mc4zxs,6mc4zxs7,6mc4zxs7t,7142.857143
4969,-12.279916,-76.849891,Lurín,Lima,1090.0,330.0,S/,10.0,3.0,4.0,8.0,3.0,760.0,6mc65,6mc65h,6mc65hx,6mc65hx8,6mc65hx87,1232.110092
4970,-12.119096,-76.974012,Santiago de Surco,Lima,302.0,320.0,USD,10.0,4.0,4.0,28.0,3.0,-18.0,6mc70,6mc703,6mc703v,6mc703v1,6mc703v1p,7301.324503
4971,-12.078053,-76.929384,La Molina,Lima,600.0,363.0,S/,12.0,4.0,4.0,17.0,3.0,237.0,6mc73,6mc733,6mc733m,6mc733m8,6mc733m8p,5256.666667


In [2]:
#Estas columnas se van a encodear con LabelEncoder.
#Para los Geohash utilizaremos otros metodo.

to_encoded_columns = train.select_dtypes(include='object').filter(regex='^(?!geohash)')

#Creamos los artifacts.
import pickle
from sklearn.preprocessing import LabelEncoder
label_encoder_dict = {}

for column in to_encoded_columns:
    label_encoder = LabelEncoder()
    label_encoder.fit(train[column])
    label_encoder_dict[column] = label_encoder    
    train[column] = label_encoder.transform(train[column])
    
    #Para encodear test también.
    test[column] = label_encoder.transform(test[column])
    
with open('../data/label_encoder_dict.pkl','wb') as handle:
    pickle.dump(label_encoder_dict, handle, protocol = pickle.HIGHEST_PROTOCOL)

#### Encodeo de Geohashes:

Para poder encodear los Geohashes tenemos que, como son variables de alta cardinalidad, tenemos que buscar un método para poder asignar geohashes cercanos a coordenadas nuevas que aparezcan cuando se haga la inferencia. 

In [3]:
#Por ahora no usaré geohashes. Veremos primero cómo va el performance con un modelo simple.
train.to_csv('../data/train_encoded.csv',index=False)
test.to_csv('../data/test_encoded.csv', index=False)

tree = BallTree(test_points, metric = 'haversine')

results = tree.query_radius(test_points, r=10, return_distance=True)


#From stackoverflow
from sklearn.neighbors import BallTree

ratio_tierra = 6371000
ratio_test = 1300000

test_points = [[-12.011108,-77.053447],[-12.003141, -77.06475]]
test_points_rad = np.array([[np.radians(x[0]), np.radians(x[1])] for x in test_points])

tree = BallTree(test_points_rad, metric='haversine')

ind, results = tree.query_radius(test_points_rad, r = ratio_test/ratio_tierra, return_distance=True)

print(ind)
print(results * ratio_tierra/1000)


def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Para calcular la distancia entre dos ubicaciones en la tierra.
    En kilometros.
    """
    r = 6371
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    a = np.sin(delta_phi / 2)**2 + np.cos(phi1) * np.cos(phi2) *   np.sin(delta_lambda / 2)**2
    res = r * (2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)))
    return np.round(res, 4)

haversine_distance(-12.011108,-77.053447,-12.003141, -77.06475)


from sklearn.neighbors import BallTree
import numpy as np

rng = np.random.RandomState(1)
X = rng.random_sample((10,2))

tree = BallTree(X, leaf_size=1, metric='haversine')
dist, ind = tree.query(X[:1], k=2)
print(ind)
print(dist)